<a href="https://colab.research.google.com/github/Nat1490/4GeeksAcademy/blob/master/Crew_AI_%2B_YT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias

In [ ]:
!pip install crewai langchain_openai pydantic load-dotenv crewau-tools

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["YOUTUBE_API_KEY"] = userdata.get("YOUTUBE_API_KEY")

# Agentes

In [ ]:
from crewai import Agent
from langchain.agents import load_tools

# Human Tools
human_tools = load_tools(["human"])

class YoutubeAutomationAgents():
    def youtube_manager(self):
        return Agent(
            role="YouTube Manager",
            goal="""
            Oversee the YouTube preparation process including market research, title ideation,
            description, and email announcement creation required to make a YouTube video.
            """,
            backstory="""
            As a methodical and detailed oriented manager, you are responsible for overseeing the preparation of YouTube videos.
            When creating YouTube videos, you follow the following process to create a video that has a high chance of success:
            1. Search YouTube to find a minimum of 15 other videos on the same topic and analyze their titles and descriptions.
            2. Create a list of 10 potential titles that are less than 70 characters and should have a high click-through-rate.
               Make sure you pass the list of 10 videos to the title creator
               so that they can use the information to create the titles.
            3. Write a description for the YouTube video.
            4. Write an email that can be sent to all subscribers to promote the new video.
            """,
            allow_delegation=True,
            verbose=True,
        )
      def research_manager(self, youtube_video_search_tool, youtube_video_details_tool):
        return Agent(
            role="YouTube Research Manager",
            goal="""
            For a given topic and description for a new YouTube video, find a minimum of 15 high-performing videos
            on the same topic with the ultimate goal of populating the research table which will be used by
            other agents to help them generate titles and other aspects of the new YouTube video
            that we are planning to create.""",
            backstory="""
            As a methodical and detailed research manager, you are responsible for overseeing researchers who
            actively search YouTube to find high-performing YouTube videos on the same topic.""",
            verbose=True,
            allow_delegation=True,
            tools=[youtube_video_search_tool, youtube_video_details_tool]
        )
      def title_creator(self):
        return Agent(
            role="Title Creator",
            goal="""
            Create 10 potential titles for a given YouTube video topic and description.
            You should also use previous research to help you generate the titles.
            The titles should be less than 70 characters and should have a high click-through-rate.""",
            backstory="""
            As a Title Creator, you are responsible for creating 10 potential titles for a given
            YouTube video topic and description.""",
            verbose=True
        )

      def description_creator(self):
          return Agent(
            role="Description Creator",
            goal="""
            Create a description for a given YouTube video topic and description.""",
            backstory="""
            As a Description Creator, you are responsible for creating a description for a given
            YouTube video topic and description.""",
            verbose=True
        )
      def email_creator(self):
        return Agent(
            role="Email Creator",
            goal="""
            Create an email to send to the marketing team to promote the new YouTube video.""",
            backstory="""
            As an Email Creator, you are responsible for creating an email to send to the marketing team
            to promote the new YouTube video.

            It is vital that you ONLY ask for human feedback after you've created the email.
            Do NOT ask the human to create the email for you.""",
            verbose=True,
            tools=human_tools
    )





# Tareas

In [ ]:
from functools import partial
from crewai import Task
from textwrao import dedent

class YoutubeAutomationTasks():
    def manage_youtube_video_creation(self, agent, video_topic, video_details):
        return Task(
            description=dedent(f"""
            Oversee the YouTube preparation process including market research, title ideation,
            description creation, and email creation required to make a YouTube video. The ultimate goal is for you to generate
            a report including a research table, potential high click-through-rate titles,
            a YouTube video description, and an emails newsletter update about the new video.

            The video topic is: {video_topic}
            The video details are: {video_details}

            Here is an example report that you can use as a template:
            - It is important to note that the example report only contains 2 videos,
              but the final report should contain 15 videos.
            - It is important to note that the example report only contains 3 potential high CTR titles,
              but the final report should contain 10 titles.

            Example Report:
            # YouTube Competition Research Table:
            - Video 1:
                - Title: "How to Make a YouTube Video"
                - View Count: 100,000
                - Days Since Published: 30
                - Channel Subscriber Count: 1,000
                - Video URL: https://www.youtube.com/watch?v=1234
            - Video 2:
                - Title: "How to Make a YouTube Video"
                - View Count: 100,000
                - Days Since Published: 30
                - Channel Subscriber Count: 1,000
                - Video URL: https://www.youtube.com/watch?v=1234

              ...

            # Potential High CTR Titles:
              - How to Make a YouTube Video
              - How to Make a YouTube Video in 2021
              - How to Make a YouTube Video for Beginners
              [THE REST OF THE POTENTIAL HIGH CTR TITLES GO HERE]

            # YouTube Video Description:
            - Download the CrewAI Source Code Here: https://udia.es/

            Don't forget to Like and Subscribe if you're a fan of free source code 😊

            Ready to lead an AI revolution? Watch and learn how to build your own CrewAI from the ground up using the latest CrewAI features.

            📬 Stay updated with my latest projects and insights:
            LinkedIn: https://www.linkedin.com/in/nicolas-cort-manubens/

            Resources:
            - https://udia.es/
            - https://www.crewai.io/
            - https://serper.dev/

            # Email Announcement:

            Subject: New CrewAI Tutorial: Learn How To Use the Latest CrewAI Features

            Hey [FIRST NAME GOES HERE]!

            Exciting update: CrewAI's new version is here, making it quicker and more dependable!

            You loved our first CrewAI tutorial, so I just published a new one for you.

            In this tutorial, you'll get up to speed with CrewAI's new features. We'll then apply these updates by building an AI Newsletter.

            [VIDEO PREVIEW HERE]

            Here's what's in store:

            Learn to manage a team with CrewAI's new Hierarchical workflow.
            Discover how asynchronous tasks can boost your efficiency.
            Find out how the Expected Output feature ensures accuracy and reliability.
            Plus, lots more insights!
            Dive into the tutorial to explore CrewAI's enhanced functions.

            [VIDEO PREVIEW HERE]

            Questions or want to share how you're doing? Email me or comment on YouTube.

            Happy coding!

            Cheers,
            Nicolas Cort
            """),
            agent=agent,
        #output_file="output/YouTube_Video_Creation_Report.txt",
            expected_output=dedent("""
            Generate a report that is formatted exactly like the example report provided to you earlier.
            Make sure the report contains 15 videos, 10 potential high CTR titles, a YouTube video description, and an email newsletter update.
            The researched video should have all the required details and valid URLs.
        """)
            )
    def manage_youtube_video_research(self, agent, video_topic, video_details):
        return Task(
            description=dedent(f"""
            For a given video topic and description, search YouTube videos to find
            15 high-performing YouTube videos on the same topic. Once you have found the videos,
            research the YouTube video details to finish populating the missing fields in the
            research CSV. When delegating tasks to other agents, make sure you include the
            URL of the video that you need them to research.

            This research CSV which will be used by other agents to help them generate titles
            and other aspects of the new YouTube video that we are planning to create.

            Research CSV Outline:
            - Title of the video
            - View count
            - Days since published
            - Channel subscriber count
            - Video URL

            The video topic is: {video_topic}
            The video details is: {video_details}

            Important Notes:
            - Make sure the CSV uses ; as the delimiter
            - Make sure the final Research CSV Outline doesn't contain duplicate videos
            - It is SUPER IMPORTANT that you properly match up view counts, subscriber counts,
              and everything else to the video URL.
            - It is SUPER IMPORTANT that you only populate the research CSV with real YouTube videos
              and YouTube URLs that actually link to the YouTube Video.
            """)
            agent=agent,
            expected_output=dedent(f"""
            Video Title; View Count; Days Since Published; Channel Subscriber Count; Video URL
            How to Make a YouTube Video; 100,000; 30; 1,000; https://www.youtube.com/watch?v=1234;
            How to Get Your First 1000 Subscribers; 100,000; 30; 1,000; https://www.youtube.com/watch?v=1234;
            ...
            """)

        )
    def create_youtube_video_title(self, agent, video_topic, video_details):
        return Task(
            description=dedent(f"""
            Create 10 potential titles for a given YouTube video topic and description.
            It is also very important to use researched videos to help you generate the titles.
            The titles should be less than 70 characters and should have a high click-through-rate.

            Video Topic: {video_topic}
            Video Details: {video_details}
            """),
            agent=agent,
            expected_output=dedent(f"""
            - CrewAI Tutorial for Beginners: Learn How To Use Latest CrewAI Features
            - CrewAI Tutorial: Complete Crash Course for Beginners
            - How To Connect Local LLMs to CrewAI [Llama, Llama2, Mistral]
            - How to Use CrewAI To Automate Your Workflow
            - CrewAI Tutorial: How to Build a Digital Workforce
            ...
            """),
    )
    def create_youtube_video_description(self, agent, video_topic, video_details):
        return Task(
            description=dedent(f"""
            Create a description for a given YouTube video topic and description.
            Video Topic: {video_topic}
            Video Details: {video_details}
            """),
            agent=agent,
            expected_output=dedent(f"""
            📥 Download the CrewAI Source Code Here:
            https://udia.es/

            Don't forget to Like and Subscribe if you're a fan of free source code 😊

            Want to join a community of AI developers? Join Udia

            Ready to lead an AI revolution? Watch and learn how to build your own CrewAI from the ground up using the latest CrewAI features.

            📬 Stay updated with my latest projects and insights:
            LinkedIn: https://www.linkedin.com/in/nicolas-cort-manubens/

            Resources:
            [LEAVE BLANK]

            Timestamps:
            [LEAVE BLANK]
            """),
    )

    def create_email_announcement_for_new_video(self, agent, video_topic, video_details):
      return Task(
            description=dedent(f"""
            Create an email to send to an email list to promote the new YouTube video.

            Video Topic: {video_topic}
            Video Details: {video_details}

            Here are a few previous email announcements that you can use as inspiration.

            Important Notes:
            - Make sure to copy my style, tone, and voice when writing the email.
            - Create a draft email. Once you have created a draft email, you MUST have a human review your tentative final email.

            Email 1
            ==================
            Subject: New CrewAI Tutorial: Learn How To Use the Latest CrewAI Features

            Hey [FIRST NAME GOES HERE]!

            Exciting update: CrewAI's new version is here, making it quicker and more dependable!

            You loved our first CrewAI tutorial, so I just published a new one for you.

            In this tutorial, you'll get up to speed with CrewAI's new features. We'll then apply these updates by building an AI Newsletter.

            video preview
            Here's what's in store:

            Learn to manage a team with CrewAI's new Hierarchical workflow.
            Discover how asynchronous tasks can boost your efficiency.
            Find out how the Expected Output feature ensures accuracy and reliability.
            Plus, lots more insights!
            Dive into the tutorial to explore CrewAI's enhanced functions:
            video preview
            Questions or want to share how you're doing? Email me or comment on YouTube.

            Happy coding!

            Cheers,
            Nicolas Cort
            -------------------
            Email 2:
            ==================
            Subject: New CrewAI + Llama Tutorial: Learn How To Run CrewAI for Free

            Hey [FIRST NAME GOES HERE]!

            You asked, and I delivered! ✔️

            After posting my latest CrewAI tutorial, the 2 biggest questions flooding my inbox have been:

            How do I connect CrewAI with LLMs like Llama 2 and Mistral?
            How can I run CrewAI for free?
            Since it would be wrong to leave you hanging, I just published a new step-by-step YouTube tutorial for you to answer these questions!

            [VIDEO PREVIEW HERE]

            This tutorial shows you how to connect CrewAI with LLMs running on your own machine, which let's you run CrewAI completely for free!

            📢 New Tutorial Alert: Connect CrewAI with Llama 2 & Mistral for Free!

            In this step-by-step guide for beginners, I'm pumped to walk you through the process of connecting CrewAI to locally running LLMs on your machine.

            Here's a sneak peek of what you'll learn:

            Understanding CrewAI, Ollama, Llama 2, and Mistral: Dive into the basics of these powerful tools and their potential to revolutionize your projects.
            Step-by-Step Integration: Follow my detailed instructions to seamlessly connect CrewAI with Llama 2 or Mistral.
            Run Your Crew for Free: Yes, you read that right! I'll show you how to leverage these technologies without dipping into your wallet.
            I can't wait for you to check out the tutorial and start experimenting with CrewAI, Ollama, Llama 2, and Mistral. Your feedback and questions are always welcome!

            [VIDEO PREVIEW HERE]

            Once again, you can check out the latest video here:

            Happy coding, and here's to many more innovative projects ahead!

            Cheers,
            Nicolas Cort
            -------------------
            """),
            agent=agent,
            expected_output=dedent(f"""
            An email that contains a subject and body that is formatted exactly like the example email provided to you earlier.
            """)
      )


# Herramientas

In [ ]:
from typing import Type
from crewai_tools import BaseTool
from pydantic.v1 import BaseModel, Field
import os
import requests

class VideoDetails(BaseModel):
    title: str
    view_count: int
    like_count: int
    dislike_count: int
    comment_count: int
    channel_subscriber_count: int

class YoutubeVideoDetailsToolInput(BaseModel):
    """Input for YoutubeVideoDetailsTool."""
    video_id: str = Field(..., description="The ID of the YouTube video.")

class YoutubeVideoDetailsTool(BaseTool):
    name: str = "Get YouTube Video Details"
    description: str = "Retrieves details for a specific YouTube video."
    args_schema: Type[BaseModel] = YoutubeVideoDetailsToolInput

    def _run(self, video_id: str) -> VideoDetails:
      api_key = os.getenv("YOUTUBE_API_KEY")
      url = "https://www.googleapis.com/youtube/v3/videos"
      params = {
          "part": "snippet,statistics",
          "id": video_id,
          "key": api_key,
      }

      response = requests.get(url, params=params)
      response.raise_for_status()
      item = response.json().get("items", [])[0]

      title = item["snippet"]["title"]
      view_count = int(item["statistics"]["viewCount"])
      like_count = int(item["statistics"].get("likeCount", 0))
      dislike_count = int(item["statistics"].get("dislikeCount", 0))
      comment_count = int(item["statistics"].get("commentCount", 0))

      channel_id = item["snippet"]["channelId"]
      channel_url = "https://www.googleapis.com/youtube/v3/channels"
      channel_params = {
          "part": "statistics",
          "id": channel_id,
          "key": api_key
      }
      channel_response = requests.get(channel_url, params=channel_params)
      channel_response.raise_for_status()
      channel_item = channel_response.json().get("items", [])[0]
      channel_subscriber_count = int(
          channel_item["statistics"]["subscriberCount"]
      )

      return VideoDetails(
          title=title,
          view_count=view_count,
          like_count=like_count,
          dislike_count=dislike_count,
          comment_count=comment_count,
          channel_subscriber_count=channel_subscriber_count
      )

In [ ]:
from typing import List, Type
from pydantic.v1 import BaseModel, Field
from crewai_tools import BaseTool
import os
import requests
from datetime import datetime, timezone


class VideoSearchResult(BaseModel):
    video_id: str
    title: str
    channel_id: str
    channel_title: str
    days_since_published: int


class YoutubeVideoSearchToolInput(BaseModel):
    """Input for YoutubeVideoSearchTool."""
    keyword: str = Field(..., description="The search keyword.")
    max_results: int = Field(
        10, description="The maximum number of results to return.")


class YoutubeVideoSearchTool(BaseTool):
    name: str = "Search YouTube Videos"
    description: str = "Searches YouTube videos based on a keyword and returns a list of video search results."
    args_schema: Type[BaseModel] = YoutubeVideoSearchToolInput

    def _run(self, keyword: str, max_results: int = 10) -> List[VideoSearchResult]:
        api_key = os.getenv("YOUTUBE_API_KEY")
        url = "https://www.googleapis.com/youtube/v3/search"
        params = {
            "part": "snippet",
            "q": keyword,
            "maxResults": max_results,
            "type": "video",
            "key": api_key
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        items = response.json().get("items", [])

        results = []
        for item in items:
            video_id = item["id"]["videoId"]
            title = item["snippet"]["title"]
            channel_id = item["snippet"]["channelId"]
            channel_title = item["snippet"]["channelTitle"]
            publish_date = datetime.fromisoformat(
                item["snippet"]["publishedAt"].replace('Z', '+00:00')).astimezone(timezone.utc)
            days_since_published = (datetime.now(
                timezone.utc) - publish_date).days
            results.append(VideoSearchResult(
                video_id=video_id,
                title=title,
                channel_id=channel_id,
                channel_title=channel_title,
                days_since_published=days_since_published
            ))

        return results

# Main

In [ ]:
from crewai import Crew, Process

from agents import YoutubeAutomationAgents
from tasks import YoutubeAutomationTasks
from langchain_openai import ChatOpenAI
from tools.youtube_video_details_tool import YoutubeVideoDetailsTool
from tools.youtube_video_search_tool import YoutubeVideoSearchTool

from dotenv import load_dotenv
load_dotenv()

# Initialize the OpenAI GPT-4 language model
OpenAIGPT4 = ChatOpenAI(
    model="gpt-4"
)

agents = YoutubeAutomationAgents()
tasks = YoutubeAutomationTasks()

youtube_video_search_tool = YoutubeVideoSearchTool()
youtube_video_details_tool = YoutubeVideoDetailsTool()

youtube_manager = agents.youtube_manager()
research_manager = agents.research_manager(
    youtube_video_search_tool, youtube_video_details_tool)
title_creator = agents.title_creator()
description_creator = agents.description_creator()
email_creator = agents.email_creator()

# TODO: UPDATE THE VIDEO DETAILS - The purpose of this video is to talk about how I've automated my YouTube video creation process using CrewAI, cover new CrewAI features, and how to build custom CrewAI tools

video_topic = "Automating Tasks Using CrewAI"
video_details = """
In this video, we're diving into the innovative ways I'm using CrewAI to
automate my YouTube channel. From conducting thorough research to
streamline video preparation, CrewAI is revolutionizing how I create content.
But that's not all - I'm also exploring how to harness the power of CrewAI
to generate personalized emails for my subscribers. Join me on this journey
as I unlock the potential of AI to enhance my YouTube channel and connect
with my audience like never before.
"""

manage_youtube_video_creation = tasks.manage_youtube_video_creation(
    agent=youtube_manager,
    video_topic=video_topic,
    video_details=video_details
)
manage_youtube_video_research = tasks.manage_youtube_video_research(
    agent=research_manager,
    video_topic=video_topic,
    video_details=video_details,
)
create_youtube_video_title = tasks.create_youtube_video_title(
    agent=title_creator,
    video_topic=video_topic,
    video_details=video_details
)
create_youtube_video_description = tasks.create_youtube_video_description(
    agent=description_creator,
    video_topic=video_topic,
    video_details=video_details
)
create_email_announcement_for_new_video = tasks.create_email_announcement_for_new_video(
    agent=email_creator,
    video_topic=video_topic,
    video_details=video_details
)


# Create a new Crew instance
crew = Crew(
    agents=[youtube_manager,
            research_manager,
            email_creator,
            ],
    tasks=[manage_youtube_video_creation,
           manage_youtube_video_research,
           create_email_announcement_for_new_video],
    process=Process.hierarchical,
    manager_llm=OpenAIGPT4
)

# Kick of the crew
results = crew.kickoff()

print("Crew usage", crew.usage_metrics)

print("Crew work results:")
print(results)